## Question 1

In [23]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

In [32]:


source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())
print('soup ready')

soup ready


In [35]:
table1 = soup.find('div', class_ ='mw-parser-output').table
table_row = table1.find_all('td')

column_names = ['PostalCode', 'Borough']
df = pd.DataFrame(columns=column_names)

for td in table_row:
    boroughNeigh = td.p.span.text
    if boroughNeigh != "Not assigned":
        pstcd = td.p.b.text
        boroughNeigh = td.p.span.text
        df = df.append({'PostalCode':pstcd, 'Borough':boroughNeigh}, ignore_index=True)
df.head(10)

,PostalCode,Borough
0,M3A,North York(Parkwoods)
1,M4A,North York(Victoria Village)
2,M5A,Downtown Toronto(Regent Park / Harbourfront)
3,M6A,North York(Lawrence Manor / Lawrence Heights)
4,M7A,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke(Islington Avenue)
6,M1B,Scarborough(Malvern / Rouge)
7,M3B,North York(Don Mills)North
8,M4B,East York(Parkview Hill / Woodbine Gardens)
9,M5B,"Downtown Toronto(Garden District, Ryerson)"


In [36]:
df = df[df.PostalCode != 'M7A'].reset_index(drop=True)

#Spliting the Borough column into Borough & Neighbourhood
df["Neighborhood"] = df["Borough"].str.split(pat='(', n=-1, expand=True)[1]
df["Neighborhood"] = df["Neighborhood"].str.split(pat=')', n=-1, expand=True)[0]
df["Borough"] = df["Borough"].str.split(pat='(', n=-1, expand=True)[0]

#If there are multiple neighborhoods replace the separator  '/' with ','
df['Neighborhood'] = df['Neighborhood'].str.replace(' /', ',', n=-1)

In [37]:
## Grouping the data
df['Borough'].value_counts()

North York                                                      24
Scarborough                                                     17
Downtown Toronto                                                17
Etobicoke                                                       11
Central Toronto                                                  9
West Toronto                                                     6
York                                                             5
East York                                                        4
East Toronto                                                     4
East YorkEast Toronto                                            1
EtobicokeNorthwest                                               1
MississaugaCanada Post Gateway Processing Centre                 1
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
Name: Borough, dtype: int64

In [38]:
df['Borough'].loc[34] = 'East York'
df['Borough'].loc[91] = 'Downtown Toronto'
df['Borough'].loc[75] = 'Mississauga'
df['Borough'].loc[99] = 'East Toronto'
df['Borough'].loc[93] = 'Etobicoke'
df['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64

In [41]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M9A,Etobicoke,Islington Avenue


In [42]:
df.shape

(102, 3)

## Question 2

In [ ]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import time

torontodata = df # creating a duplicate of df datafarame as torontodata to work with

latList = []
lngList = []

for ind in torontodata.index:
    geolocator = Nominatim(user_agent="toronto_explorer")
    location = None
    location = geolocator.geocode(torontodata['Neighborhood'][ind] + ', ' + torontodata['Borough'][ind])
    time.sleep(1)
    if location is None:
        i=0
        while (location is None):
            try:
                tempneighbor = torontodata["Neighborhood"][ind].split(', ', -1)[i] + ', ' + torontodata['Borough'][ind]
            except:
                tempneighbor = torontodata["Borough"][ind]
            
            location = geolocator.geocode(tempneighbor)
            time.sleep(1)
            i += 1
    latList.append(location.latitude)
    lngList.append(location.longitude)
    
torontodata['Latitude'] = latList 
torontodata['Longitude'] = lngList

torontodata.head(10)